In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Loading image data

In [0]:
import bz2, pickle

with bz2.BZ2File('/content/drive/My Drive/Colab Notebooks/test_label_128.pbz2','r') as f:
    test_label = pickle.load(f)
with bz2.BZ2File('/content/drive/My Drive/Colab Notebooks/train_label_128.pbz2','r') as f:
    train_label = pickle.load(f)
with bz2.BZ2File('/content/drive/My Drive/Colab Notebooks/test_data_128.pbz2','r') as f:
    test_data = pickle.load(f)
with bz2.BZ2File('/content/drive/My Drive/Colab Notebooks/train_data_1_128.pbz2','r') as f:
    train_data1 = pickle.load(f)
    
print(test_label.shape)
print(test_data.shape)
print(train_label.shape)
print(train_data1.shape)
#print(train_data2.shape)

(202, 14)
(202, 128, 128, 1)
(191027, 14)
(30000, 128, 128, 1)


## Creating Attention Model

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.applications.densenet import *
from keras.applications.vgg16 import *
from keras.optimizers import *
from keras.layers import *
from keras.optimizers import Adam, SGD
from keras.models import Model, load_model
from keras.activations import *

import keras, tensorflow as tf, keras.backend as K, numpy as np, cv2

Using TensorFlow backend.


In [0]:
base_pretrained_model = VGG16(weights=None,include_top=False, input_shape=(128,128,1))
base_pretrained_model.trainable = False
in_lay = Input((128,128,1))
pt_depth = base_pretrained_model.get_output_shape_at(0)[-1]
pt_features = base_pretrained_model(in_lay)
bn_features = BatchNormalization()(pt_features)

# here we do an attention mechanism to turn pixels in the GAP on an off

attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(bn_features)
attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
attn_layer = LocallyConnected2D(1, kernel_size = (1,1), padding = 'valid', activation = 'sigmoid')(attn_layer)
# fan it out to all of the channels
up_c2_w = np.ones((1, 1, 1, pt_depth))
up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', activation = 'linear', use_bias = False, weights = [up_c2_w])
up_c2.trainable = False
attn_layer = up_c2(attn_layer)

mask_features = multiply([attn_layer, bn_features])
gap_features = GlobalAveragePooling2D()(mask_features)
gap_mask = GlobalAveragePooling2D()(attn_layer)
# to account for missing values from the attention model
gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
gap_dr = Dropout(0.5)(gap)
dr_steps = Dropout(0.25)(Dense(256, activation = 'relu')(gap_dr))
out_layer = Dense(14, activation = 'sigmoid')(dr_steps) # linear is what 16bit did
att_model = Model(inputs = [in_lay], outputs = [out_layer])

att_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
att_model.summary()













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   (None, 4, 4, 512)    14713536    input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 4, 4, 512)    2048        vgg16[1][0]                      
___________________________________________________________________

## Training Attention model

In [0]:
att_model.fit(train_data1[:20000], train_label[:20000], validation_data = (train_data1[20000:25000], train_label[20000:25000]), epochs=15, batch_size=64)



Train on 20000 samples, validate on 5000 samples
Epoch 1/15
20000/20000 [==============================] - 30s 1ms/step - loss: 0.3784 - acc: 0.8346 - val_loss: 0.3576 - val_acc: 0.8435
Epoch 2/15
20000/20000 [==============================] - 24s 1ms/step - loss: 0.3566 - acc: 0.8402 - val_loss: 0.3508 - val_acc: 0.8447
Epoch 3/15
20000/20000 [==============================] - 24s 1ms/step - loss: 0.3484 - acc: 0.8471 - val_loss: 0.3458 - val_acc: 0.8490
Epoch 4/15
20000/20000 [==============================] - 24s 1ms/step - loss: 0.3458 - acc: 0.8492 - val_loss: 0.3455 - val_acc: 0.8498
Epoch 5/15
20000/20000 [==============================] - 24s 1ms/step - loss: 0.3441 - acc: 0.8506 - val_loss: 0.3434 - val_acc: 0.8498
Epoch 6/15
20000/20000 [==============================] - 24s 1ms/step - loss: 0.3432 - acc: 0.8512 - val_loss: 0.3424 - val_acc: 0.8506
Epoch 7/15
20000/20000 [==============================] - 24s 1ms/step - loss: 0.3422 - acc: 0.8512 - val_loss: 0.3417 - val_ac

## Testing Attention model

In [0]:
test_loss, test_accu = att_model.evaluate(train_data1[25000:30000,], train_label[25000:30000,])
print('Test Accuracy =', test_accu*100, '%')

5000/5000 [==============================] - 5s 1ms/step
Test Accuracy = 85.24714132308961 %
